<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=05f8a860123ea685680de02e6c765394f32f157cbbd1d98d2b514cb867df9970
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-17 15:17:28
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15


-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  66.76 K
Current:  1.39 C
-------------------
Today PnL: -1.01 L (-0.72%)
Current PnL: -27.75 L (-17.93%)
CY Booked + Current PnL: -13.46 L (-8.7%)
-------------------
Total profit:  1.30 L
Total loss:  -29.05 L
-------------------
Total Booked + Current PnL: 13.52 L (10.58%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.26%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.32 C
Est FTT PnL: 92.39 L (66.32%)
Deployed:  1.28 C
Current:  1.39 C
CAGR/XIRR %: 4.77%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,-1.42,-19.24,23.94,0.09,19483.0,-19393.0,81384.0,88.90,30.0,M-SC,2.45,253.0,-1.00,0.59,6.92,OX40N,NTT,DURABLES
82,VOLTAS,1530.00,-1.00,7.98,10.85,19.69,22464.0,15303.0,207045.0,-2.60,54.0,X-MC,2.54,78.0,0.68,1.49,15.32,XY25,NTT,AC
50,MASFIN,398.61,-0.72,-3.49,26.46,22.05,25021.0,-3420.0,94560.0,-16.89,50.0,H-SC,6.87,164.0,-0.14,0.68,37.45,XR,ATH,FINANCE
39,INDIGOPNTS,1408.00,-1.39,-14.61,17.13,0.02,25534.0,-25499.0,149060.0,127.12,46.0,M-SC,14.37,234.0,-1.00,1.08,29.66,OX40N,NTT,PAINTS
43,ITC,452.00,-0.47,-2.01,13.06,10.78,30754.0,-4836.0,235482.0,-41.56,42.0,X-LC,2.00,5.0,-0.16,1.70,3.12,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,2.66,-14.60,112.73,81.67,90313.0,-13696.0,80114.0,7200.00,52.0,L-SC,12.12,271.0,-0.15,0.58,55.14,XR,NTT,CERAMICS
13,BSOFT,831.70,1.29,-21.75,92.70,50.79,102024.0,-30595.0,110058.0,0.50,65.0,H-SC,6.44,151.0,-0.30,0.79,28.14,XR,ATH,IT
63,SFL,1287.00,1.05,-43.71,124.96,26.63,184443.0,-114616.0,147602.0,11.76,30.0,M-SC,11.87,239.0,-0.62,1.06,1.05,XY24,NTT,MISC
67,SONACOMS,806.63,1.02,-15.25,64.65,39.54,55426.0,-15427.0,85732.0,-33.70,51.0,M-SC,4.42,220.0,-0.28,0.62,20.89,AR,ATH,AUTO
38,INDIAMART,4810.62,0.98,-3.61,114.46,106.72,136075.0,-4452.0,118884.0,-52.34,38.0,H-SC,3.28,139.0,-0.03,0.86,18.92,AR,ATH,MISC


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COFFEEDAY,80.00,-3.77,-43.02,137.39,35.27,88897.0,-48845.0,64704.0,-55.73,42.0,L-SC,10.71,270.0,-0.55,0.47,58.36,XR,NTT,HOTELS
19,COLPAL,3726.84,-3.27,-20.65,78.33,41.51,163694.0,-54385.0,208980.0,-5.42,35.0,X-MC,7.38,61.0,-0.33,1.51,0.00,XY25,ATH,FMCG
71,TANLA,1943.92,-3.01,-37.63,252.22,119.69,425952.0,-101880.0,168881.0,-37.17,40.0,H-SC,7.99,156.0,-0.24,1.22,33.42,AR,ATH,IT
15,CAMS,950.00,-2.63,-3.83,29.43,24.48,71285.0,-9636.0,242220.0,-81.76,37.0,X-SC,0.23,86.0,-0.14,1.75,18.43,X40N,NTT,MISC
22,DIXON,18931.72,-2.52,-11.53,42.50,26.07,73400.0,-22516.0,172705.0,-56.97,37.0,X-MC,9.67,56.0,-0.31,1.25,7.56,X40N,ATH,IT


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-1.93,-3.17,114.71,107.91,164002.0,-4673.0,142971.0,-21.36,53.0,M-SC,12.58,216.0,-0.03,1.03,6.60,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-1.39,-14.61,17.13,0.02,25534.0,-25499.0,149060.0,127.12,46.0,M-SC,14.37,234.0,-1.00,1.08,29.66,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,-1.42,-19.24,23.94,0.09,19483.0,-19393.0,81384.0,88.90,30.0,M-SC,2.45,253.0,-1.00,0.59,6.92,OX40N,NTT,DURABLES
66,SIS,528.00,-0.03,-24.20,59.98,21.26,50481.0,-26869.0,84163.0,1985.49,50.0,H-SC,5.06,166.0,-0.53,0.61,13.71,OX40N,NTT,MISC
47,KANSAINER,340.00,-1.14,-25.92,53.17,13.47,106219.0,-69894.0,199773.0,-69.48,32.0,H-SC,5.68,158.0,-0.66,1.44,1.58,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,0.77,7.14,60.78,72.27,110345.0,12100.0,181548.0,-7.89,66.0,M-LC,2.61,99.0,0.11,1.31,13.76,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,0.77,7.14,60.78,72.27,110345.0,12100.0,181548.0,-7.89,66.0,M-LC,2.61,99.0,0.11,1.31,13.76,XR,NTT,IT
25,FINCABLES,1641.55,-1.93,-3.17,114.71,107.91,164002.0,-4673.0,142971.0,-21.36,53.0,M-SC,12.58,216.0,-0.03,1.03,6.60,OX40N,ATH,CABLES
38,INDIAMART,4810.62,0.98,-3.61,114.46,106.72,136075.0,-4452.0,118884.0,-52.34,38.0,H-SC,3.28,139.0,-0.03,0.86,18.92,AR,ATH,MISC
85,ZYDUSLIFE,1286.17,-0.79,-3.58,40.66,35.63,82162.0,-7503.0,202071.0,-17.18,35.0,H-MC,2.93,119.0,-0.09,1.46,12.50,AR,ATH,PHARMA
37,IEX,219.00,-0.19,-4.90,56.28,48.63,107020.0,-9798.0,190156.0,-36.71,46.0,H-SC,14.08,136.0,-0.09,1.37,6.09,XR,NTT,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,-0.83,-18.92,91.76,55.47,180998.0,-46035.0,197251.0,-28.25,23.0,X-MC,13.43,64.0,-0.25,1.42,1.07,X40N,ATH,FINANCE
53,PAGEIND,51605.08,-0.91,-10.57,43.25,28.11,77904.0,-21285.0,180125.0,-35.24,26.0,X-MC,13.21,55.0,-0.27,1.30,0.00,X40,ATH,APPARELS
11,BATAINDIA,2096.00,-0.94,-39.36,122.98,35.20,95949.0,-50650.0,78020.0,2.24,29.0,X-SC,16.29,92.0,-0.53,0.56,0.00,X40,NTT,FOOTWEAR
8,AWL,485.00,-0.74,-23.63,100.99,53.50,232723.0,-71309.0,230442.0,-62.99,29.0,X-MC,3.28,58.0,-0.31,1.66,2.35,XY24,NTT,FMCG
2,ABBOTINDIA,35195.00,-0.47,-7.16,27.17,18.07,45116.0,-12805.0,166050.0,-20.44,30.0,X-MC,6.03,63.0,-0.28,1.20,10.03,X40,ATH,PHARMA


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,CAMS,950.0,-2.63,-3.83,29.43,24.48,71285.0,-9636.0,242220.0,-81.76,37.0,X-SC,0.23,86.0,-0.14,1.75,18.43,X40N,NTT,MISC
43,ITC,452.0,-0.47,-2.01,13.06,10.78,30754.0,-4836.0,235482.0,-41.56,42.0,X-LC,2.00,5.0,-0.16,1.70,3.12,X40,NTT,FMCG
20,DABUR,735.0,-0.64,-3.42,48.79,43.70,116896.0,-8478.0,239590.0,-11.44,35.0,X-MC,2.26,72.0,-0.07,1.73,11.12,XY24,BTT,FMCG
33,HINDUNILVR,2922.0,-0.23,-11.47,28.39,13.67,64613.0,-29473.0,227590.0,-18.11,31.0,X-LC,2.31,14.0,-0.46,1.64,7.44,XY25,NTT,FMCG
36,ICICIPRULI,790.0,-1.10,4.41,25.21,30.73,51854.0,8685.0,205690.0,-20.00,53.0,X-MC,2.38,75.0,0.17,1.48,17.74,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.00,-0.67,-26.05,121.98,64.15,214636.0,-61991.0,175960.0,-56.75,31.0,X-SC,2.81,82.0,-0.29,1.27,0.00,XY24,BTT,CEMENT
11,BATAINDIA,2096.00,-0.94,-39.36,122.98,35.20,95949.0,-50650.0,78020.0,2.24,29.0,X-SC,16.29,92.0,-0.53,0.56,0.00,X40,NTT,FOOTWEAR
19,COLPAL,3726.84,-3.27,-20.65,78.33,41.51,163694.0,-54385.0,208980.0,-5.42,35.0,X-MC,7.38,61.0,-0.33,1.51,0.00,XY25,ATH,FMCG
53,PAGEIND,51605.08,-0.91,-10.57,43.25,28.11,77904.0,-21285.0,180125.0,-35.24,26.0,X-MC,13.21,55.0,-0.27,1.30,0.00,X40,ATH,APPARELS
31,HAVELLS,2069.16,-1.01,-10.51,48.04,32.48,141006.0,-34474.0,293517.0,-15.05,38.0,X-MC,6.11,67.0,-0.24,2.12,0.56,X40,ATH,ELECTRICAL


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,-1.90,-30.72,106.19,42.84,47823.0,-19971.0,45035.0,-54.82,39.0,X-SC,35.39,83.0,-0.42,0.32,1.89,XY24,NTT,MISC
59,RELAXO,1176.00,-0.07,-48.10,193.49,52.31,145760.0,-69828.0,75332.0,-44.73,40.0,X-SC,7.10,91.0,-0.48,0.54,0.84,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.94,-39.36,122.98,35.20,95949.0,-50650.0,78020.0,2.24,29.0,X-SC,16.29,92.0,-0.53,0.56,0.00,X40,NTT,FOOTWEAR
51,MEDANTA,1486.00,0.32,-4.52,29.95,24.08,36303.0,-5739.0,121211.0,-12.23,33.0,X-SC,5.14,89.0,-0.16,0.87,14.21,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,-0.35,-16.77,72.20,43.32,97874.0,-27312.0,135560.0,-28.43,36.0,X-SC,4.87,90.0,-0.28,0.98,4.09,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,0.43,-11.50,36.39,20.71,121817.0,-43482.0,334755.0,-23.62,61.0,X-LC,4.24,1.0,-0.36,2.41,11.85,X40,ATH,IT
41,INFY,1972.00,0.53,9.02,23.14,34.24,78930.0,28210.0,341098.0,-14.93,61.0,X-LC,4.28,2.0,0.36,2.46,18.05,X40,NTT,IT
75,TMPV,600.00,0.19,-29.06,73.36,22.98,117555.0,-65654.0,160244.0,-26.74,30.0,X-LC,6.75,3.0,-0.56,1.16,0.79,XY24,NTT,AUTO
81,VBL,671.64,-1.19,-4.77,42.46,35.67,127713.0,-15057.0,300785.0,-16.08,51.0,X-LC,4.00,4.0,-0.12,2.17,8.43,X40N,ATH,FMCG
43,ITC,452.00,-0.47,-2.01,13.06,10.78,30754.0,-4836.0,235482.0,-41.56,42.0,X-LC,2.00,5.0,-0.16,1.70,3.12,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,2.66,-14.60,112.73,81.67,90313.0,-13696.0,80114.0,7200.00,52.0,L-SC,12.12,271.0,-0.15,0.58,55.14,XR,NTT,CERAMICS
50,MASFIN,398.61,-0.72,-3.49,26.46,22.05,25021.0,-3420.0,94560.0,-16.89,50.0,H-SC,6.87,164.0,-0.14,0.68,37.45,XR,ATH,FINANCE
13,BSOFT,831.70,1.29,-21.75,92.70,50.79,102024.0,-30595.0,110058.0,0.50,65.0,H-SC,6.44,151.0,-0.30,0.79,28.14,XR,ATH,IT
39,INDIGOPNTS,1408.00,-1.39,-14.61,17.13,0.02,25534.0,-25499.0,149060.0,127.12,46.0,M-SC,14.37,234.0,-1.00,1.08,29.66,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.25,-0.21,-2.65,24.24,20.95,59410.0,-6679.0,245089.0,-3.58,48.0,X-LC,17.12,31.0,-0.11,1.77,32.04,X40,ATH,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.70,1.29,-21.75,92.70,50.79,102024.0,-30595.0,110058.0,0.50,65.0,H-SC,6.44,151.0,-0.30,0.79,28.14,XR,ATH,IT
84,WIPRO,420.00,0.77,7.14,60.78,72.27,110345.0,12100.0,181548.0,-7.89,66.0,M-LC,2.61,99.0,0.11,1.31,13.76,XR,NTT,IT
41,INFY,1972.00,0.53,9.02,23.14,34.24,78930.0,28210.0,341098.0,-14.93,61.0,X-LC,4.28,2.0,0.36,2.46,18.05,X40,NTT,IT
73,TCS,4389.97,0.43,-11.50,36.39,20.71,121817.0,-43482.0,334755.0,-23.62,61.0,X-LC,4.24,1.0,-0.36,2.41,11.85,X40,ATH,IT
27,GLAXO,3466.20,-0.09,3.70,34.19,39.15,70653.0,7364.0,206648.0,-16.74,53.0,X-MC,10.62,60.0,0.10,1.49,33.77,X40,ATH,PHARMA


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.30
1,25,44.84
2,50,76.65


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.84
MC    30.94
LC    24.21
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.68
X40      23.07
X40N     13.01
XR        9.76
XY25      9.08
AR        9.04
OX40N     7.60
SR        0.99
MH        0.76
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.45
H-SC    23.75
X-LC    20.74
M-SC    11.66
X-SC     8.01
H-MC     4.82
L-SC     1.42
M-MC     1.33
M-LC     1.31
H-LC     1.13
L-LC     1.03
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.37,-8.33,43.16
IT,13.05,-16.20,74.62
FINANCE,9.45,-21.63,74.53
MISC,7.26,-28.91,81.64
ELECTRICAL,6.02,-12.48,53.68
PAINTS,5.86,-13.68,30.57
INSURANCE,4.78,-2.25,37.23
PHARMA,4.15,-2.25,34.44
AUTO,2.83,-33.93,80.37


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3219602.0,21
AR,1318689.0,10
XR,1307782.0,13
X40,1023369.0,14
X40N,883631.0,9
OX40N,739991.0,10
XY25,382748.0,6
SR,283819.0,2
MH,79067.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3686123.0,25
M-SC,1441591.0,15
X-MC,1437220.0,16
X-LC,850668.0,11
X-SC,807019.0,8
H-MC,411072.0,3
L-SC,264580.0,3
M-LC,110345.0,1
H-LC,76485.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1292375.0      6
           AR         901183.0      5
M-SC       XY24       827217.0      6
H-SC       XR         787446.0      7
X-MC       X40        497209.0      7
           XY24       406725.0      3
           X40N       347128.0      4
H-SC       OX40N      342233.0      4
X-LC       X40        332822.0      5
X-SC       X40N       314919.0      3
M-SC       OX40N      312388.0      5
X-SC       XY24       298762.0      3
H-SC       SR         283819.0      2
X-LC       X40N       221584.0      2
H-MC       AR         214731.0      2
X-LC       XY24       198182.0      2
H-MC       XY24       196341.0      1
X-SC       X40        193338.0      2
X-MC       XY25       186158.0      2
L-SC       XR         179210.0      2
M-SC       XR         175696.0      2
           AR         126290.0      2
M-LC       XR         110345.0      1
X-LC       XY25        98080.0      2
L-SC       OX40N       85370.0      1
H-SC       MH          79067.0      1
H-LC       AR          76485.0      1
M-MC       XY25        61402.0      1
L-MC       XR          55085.0      1
L-LC       XY25        37108.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 56.0 seconds
